In [1]:
%load_ext autoreload
%autoreload 2

2022-02-04 18:36:07 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [1]:
from lm_fairseq_transformer_wrapper import FairseqTransformerHub, parse_single_alignment
import warnings
import torch
import numpy as np

torch.cuda.set_device(1)
torch.cuda.current_device()

1

In [2]:
europarl_dir = "/home/usuaris/veu/javier.ferrando/norm-analysis-of-transformer/exp2_nmt/work/"
ckpt_dir = "/home/usuaris/veu/belen.alastruey/europarl_lm_ckpt/"
dict_dir = "/home/usuaris/veu/belen.alastruey/europarl_lm_data/data-bin"

### Alignment test (BPE from Europarl)
test_de_bpe = europarl_dir + "processed_data/test.bpe.de"
test_de_word =  europarl_dir + "processed_data/test.de"
test_en_bpe = europarl_dir + "processed_data/test.bpe.en"
test_en_word = europarl_dir + "processed_data/test.en"
gold_alignment = europarl_dir + "gold_alignment/alignment.talp"

warnings.simplefilter('ignore')

with open(test_de_bpe, encoding="utf-8") as fbpe:
    src_bpe_sents = fbpe.readlines()
with open(test_en_bpe, encoding="utf-8") as fbpe:
    tgt_bpe_sents = fbpe.readlines()
with open(europarl_dir + "data_in_progress/test.uc.de", encoding="utf-8") as fword:
    src_word_sents = fword.readlines()
with open(europarl_dir + "data_in_progress/test.uc.en", encoding="utf-8") as fword:
    tgt_word_sents = fword.readlines()


hub = FairseqTransformerHub.from_pretrained(
    ckpt_dir,
    checkpoint_file=f"checkpoint_best.pt",
    data_name_or_path=(dict_dir),
    #(data_name_or_path=iwslt14_dir / "data-bin").as_posix(),
)

2022-03-12 21:07:33 | INFO | fairseq.file_utils | Archive name '/home/usuaris/veu/belen.alastruey/europarl_lm_ckpt/' was not found in archive name list. We assumed '/home/usuaris/veu/belen.alastruey/europarl_lm_ckpt/' was a path or URL but couldn't find any file associated to this path or URL.
2022-03-12 21:07:33 | INFO | fairseq.file_utils | Archive name '/home/usuaris/veu/belen.alastruey/europarl_lm_data/data-bin' was not found in archive name list. We assumed '/home/usuaris/veu/belen.alastruey/europarl_lm_data/data-bin' was a path or URL but couldn't find any file associated to this path or URL.


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [12]:
# Test data
i = 1 # 0, 1 28, 45, 148 (pred_tok vs tgt_tok), 178, 16 (Hallucination)

src_sent, src_tok, src_tensor, tgt_sent, tgt_tok, tgt_tensor = hub.get_sample('test', i)

AttributeError: 'LanguageModelingTask' object has no attribute 'src_dict'

In [ ]:
# src_tensor = src_tensor.unsqueeze(0).to(device)
# tgt_tensor = torch.cat([
#     torch.tensor([self.task.tgt_dict.eos_index]),
#     tgt_tensor[:-1]
#     ]).unsqueeze(0).to(self.device)

# model_output, encoder_out = self.models[0](src_tensor, src_tensor.size(-1), tgt_tensor, )

# log_probs = self.models[0].get_normalized_probs(model_output, log_probs=True, sample=None)

In [13]:
model_output, log_probs, encoder_out, layer_inputs, layer_outputs = hub.trace_forward(src_tensor, tgt_tensor)

NameError: name 'src_tensor' is not defined

In [ ]:
probs = torch.nn.functional.softmax(log_probs, dim=-1)
print('probs',probs.size())
pred_ind = torch.argmax(probs,dim=-1)
print(pred_ind.size())
pred = torch.max(probs,dim=-1)
pred

